## QBUS3850 Tutorial 12: Forecasting Value at Risk and Expected Shortfall

This exercise uses a progress-bar package called 'tqdm'. The usage is informative but not essential. `pip3 install tqdm`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
import statsmodels.api as sm
import scipy
from scipy import stats
from tqdm import tqdm, trange
from arch.__future__ import reindexing

#Set print options to 4dp and suppress scientific notation
np.set_printoptions(precision=3, suppress=True) 

In [ ]:
# Import All Ordinaries Index Data
AORD_data = pd.read_csv('AllORD0017.csv', parse_dates = ['Date'], dayfirst = True)
AOdates = AORD_data['Date']
AOp = AORD_data['Close']
print( AORD_data.tail() )

## Compute log-returns 
AOr = ( 100 * ( np.log( AOp ) - np.log( AOp.shift() ) ) )[1:]
assert( np.isnan( AOr ).sum() == 0 )

## Put dates, prices and returns all on the same basis.
#AOdates = AOdates[1:]
#AOp = AOp[1:]

#Plot All Ordinaries index values and returns
fig, axs = plt.subplots(2,1, figsize = (10,8))
axs[0].plot( AOdates, AOp )
axs[0].set_title( 'AllOrds Prices' )
axs[1].plot( AOdates[1:], AOr )
axs[1].set_title( 'AllOrds log returns' )
plt.show()


# Q1 - Forecasting 1 day VaR and ES

In [ ]:
## In-sample and forecast sample
n = len(AOr)
nis = n-1000
AOris = AOr[:nis]
AOrf = AOr[nis:]

### Q1(a) Estimate GARCH(1,1) and GJR-GARCH(1,1) models, with Gaussian and Student-t errors, to the sample of log-returns leaving out the last 1000 days of returns. 

### Q1(b) Forecast 1-step-ahead VaR and ES, updating parameters every 20 days, for each of the last 1000 days of data, using the models in (a). 

### Q1(c) Include 1-step-ahead VaR and ES forecasts from a 100-day historical simulation. Include the RiskMetrics IGARCH model.


In [ ]:
def VaR_n( p ):
    return stats.norm.ppf( p )
        
def ES_n( p ):
    return -stats.norm.pdf( stats.norm.ppf( p ) ) / p

def VaR_t( p, df ):
    return stats.t.ppf( p, df ) * np.sqrt( ( df - 2 ) / df )

def ES_t( p, df ):
    return -stats.t.pdf( stats.t.ppf( p, df ), df ) / p * ( df + stats.t.ppf( p, df )**2 ) / ( df - 1 ) * np.sqrt( ( df - 2 ) / df )


In [ ]:
## 1-step-ahead VaR forecasts.
VaR1G = []
VaR5G = []
VaR1Gt = []
VaR5Gt = []
VaR1GJ = []
VaR5GJ = []
VaR1GJt = []
VaR5GJt = []
VaR1RM = []
VaR5RM = []
VaR1HS = []
VaR5HS = []

## 1-step-ahead ES forecasts.
ES1G = []
ES5G = []
ES1Gt = []
ES5Gt = []
ES1GJ = []
ES5GJ = []
ES1GJt = []
ES5GJt = []
ES1RM = []
ES5RM = []
ES1HS = []
ES5HS = []

## 1-step-ahead volatility forecasts 
SFAOg = []
SFAOgt = []
SFAOgj = []
SFAOgjt = []
SFAOrm = []

## Inferred degrees of freedom for t-GARCH models.
dfGt = []
dfGJt = []

for t in trange( nis, n ):
    ## Fixed estimation window size
    AOris = AOr.values[ t - nis : t ]

    ## GARCH models
    
    # TODO: define the 4 corresponding models   
    
    if t % 20 == nis % 20: 
        ## Re-estimate models every 20 observations
        EstMdlg = Mdlg.fit(disp='off') 
        EstMdlgt = Mdlgt.fit(disp='off')
        EstMdlgj = Mdlgj.fit(disp='off') 
        EstMdlgjt = Mdlgjt.fit(disp='off')     
    else:  
        ## Use updated dataset but previously estimated model parameters.
        EstMdlg = Mdlg.fix( params = EstMdlg.params ) 
        EstMdlgt = Mdlgt.fix( params = EstMdlgt.params ) 
        EstMdlgj = Mdlgj.fix( params = EstMdlgj.params ) 
        EstMdlgjt = Mdlgjt.fix( params = EstMdlgjt.params ) 
        
    #1-step-ahead volatility forecasts 
    SFAOg.append( np.sqrt( EstMdlg.forecast( horizon = 1 ).variance.values[-1, 0] ) )
    SFAOgt.append( np.sqrt(EstMdlgt.forecast(horizon = 1).variance.values[-1, 0 ] ) )
    SFAOgj.append( np.sqrt(EstMdlgj.forecast(horizon = 1).variance.values[-1, 0 ] ) )
    SFAOgjt.append( np.sqrt(EstMdlgjt.forecast(horizon = 1).variance.values[-1, 0 ] ) )
    
    ## Inferred degrees of freedom
    dfGt.append( EstMdlgt.params['nu'] )
    dfGJt.append( EstMdlgjt.params['nu'] )
    
    #1-day ahead VaR forecasts for GARCH-type models
    VaR1G.append( EstMdlg.params['mu'] + VaR_n(0.01) * SFAOg[-1] )
    VaR5G.append( EstMdlg.params['mu'] + VaR_n(0.05) * SFAOg[-1] )
    VaR1Gt.append( EstMdlgt.params['mu'] + VaR_t(0.01, EstMdlgt.params['nu'] ) * SFAOgt[-1] )
    VaR5Gt.append( EstMdlgt.params['mu'] + VaR_t(0.05, EstMdlgt.params['nu'] ) * SFAOgt[-1] )
    VaR1GJ.append( EstMdlgj.params['mu'] + VaR_n(0.01) * SFAOgj[-1] )
    VaR5GJ.append( EstMdlgj.params['mu'] + VaR_n(0.05) * SFAOgj[-1] )
    VaR1GJt.append( EstMdlgjt.params['mu'] + VaR_t(0.01, EstMdlgjt.params['nu'] ) * SFAOgjt[-1] )
    VaR5GJt.append( EstMdlgjt.params['mu'] + VaR_t(0.05, EstMdlgjt.params['nu'] ) * SFAOgjt[-1] )

    #1-day ahead ES forecasts for GARCH-type models
    ES1G.append( EstMdlg.params['mu'] + ES_n(0.01) * SFAOg[-1] )
    ES5G.append( EstMdlg.params['mu'] + ES_n(0.05) * SFAOg[-1] )
    ES1Gt.append( EstMdlgt.params['mu'] + ES_t(0.01, EstMdlgt.params['nu'] ) * SFAOgt[-1] )
    ES5Gt.append( EstMdlgt.params['mu'] + ES_t(0.05, EstMdlgt.params['nu'] ) * SFAOgt[-1] )
    ES1GJ.append( EstMdlgj.params['mu'] + ES_n(0.01) * SFAOgj[-1] )
    ES5GJ.append( EstMdlgj.params['mu'] + ES_n(0.05) * SFAOgj[-1] )
    ES1GJt.append( EstMdlgjt.params['mu'] + ES_t(0.01, EstMdlgjt.params['nu'] ) * SFAOgjt[-1] )
    ES5GJt.append( EstMdlgjt.params['mu'] + ES_t(0.05, EstMdlgjt.params['nu'] ) * SFAOgjt[-1] )

    ## Riskmetrics (IGARCH)
    betaRM = 0.94    
    if t == nis:
        SFAOrm.append( np.std( AOris[-200:] ) )
    else:
        SFAOrm.append( np.sqrt( ( 1 - betaRM ) * AOris[-1]**2 + betaRM * SFAOrm[-1]**2 ) )

    VaR5RM.append( SFAOrm[-1] * VaR_n(0.05) )
    VaR1RM.append( SFAOrm[-1] * VaR_n(0.01) )
    ES5RM.append( SFAOrm[-1] * ES_n(0.05) )
    ES1RM.append( SFAOrm[-1] * ES_n(0.01) )

    ## Historical Simulation
    VaR5HS.append( np.percentile( AOris[-100:], 5, interpolation = 'midpoint') )
    VaR1HS.append( np.percentile( AOris[-100:], 1, interpolation = 'midpoint') )
    x100 = AOris[-100:]
    ES5HS.append( np.mean( x100[x100<=VaR5HS[-1]] ) )
    ES1HS.append( np.mean( x100[x100<=VaR1HS[-1]] ) )

    

In [ ]:
#Plot of 5% VaRs 
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(AOdates[-1000:], AOrf, c = 'g', label = 'Returns')
axs.plot(AOdates[-1000:], VaR5G, c = 'y', label = 'GARCH')
axs.plot(AOdates[-1000:], VaR5Gt, c = 'r', label = 'GARCH-t')
axs.plot(AOdates[-1000:], VaR5GJ, c = 'r', label = 'GJR-GARCH')
axs.plot(AOdates[-1000:], VaR5GJt, c = 'cyan', label = 'GJR-GARCH-t')
axs.plot(AOdates[-1000:], VaR5HS, c = 'b', label = 'HS')
axs.plot(AOdates[-1000:], VaR5RM, c = 'k', label = 'RM')
axs.set_xlim(AOdates[-1000:].min(), AOdates[-1000:].max())
axs.set_title('5% forecast VaRs from all models with actual returns', weight = 'bold')
axs.legend()
plt.show()

#Plot of 1% VaRs 
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(AOdates[-1000:], AOrf, c = 'g', label = 'Returns')
axs.plot(AOdates[-1000:], VaR1G, c = 'y', label = 'GARCH')
axs.plot(AOdates[-1000:], VaR1Gt, c = 'r', label = 'GARCH-t')
axs.plot(AOdates[-1000:], VaR1GJ, c = 'r', label = 'GJR-GARCH')
axs.plot(AOdates[-1000:], VaR1GJt, c = 'cyan', label = 'GJR-GARCH-t')
axs.plot(AOdates[-1000:], VaR1HS, c = 'b', label = 'HS')
axs.plot(AOdates[-1000:], VaR1RM, c = 'k', label = 'RM')
axs.set_xlim(AOdates[-1000:].min(), AOdates[-1000:].max())
axs.set_title('1% forecast VaRs from all models with actual returns', weight = 'bold')
axs.legend()
plt.show()


In [ ]:
#Plot of 5% ESs 
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(AOdates[-1000:], AOrf, c = 'g', label = 'Returns')
axs.plot(AOdates[-1000:], ES5G, c = 'y', label = 'GARCH')
axs.plot(AOdates[-1000:], ES5Gt, c = 'r', label = 'GARCH-t')
axs.plot(AOdates[-1000:], ES5GJ, c = 'r', label = 'GJR-GARCH')
axs.plot(AOdates[-1000:], ES5GJt, c = 'cyan', label = 'GJR-GARCH-t')
axs.plot(AOdates[-1000:], ES5HS, c = 'b', label = 'HS')
axs.plot(AOdates[-1000:], ES5RM, c = 'k', label = 'RM')
axs.set_xlim(AOdates[-1000:].min(), AOdates[-1000:].max())
axs.set_title('5% forecast ES from all models with actual returns', weight = 'bold')
axs.legend()
plt.show()

#Plot of 1% ESs 
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(AOdates[-1000:], AOrf, c = 'g', label = 'Returns')
axs.plot(AOdates[-1000:], ES1G, c = 'y', label = 'GARCH')
axs.plot(AOdates[-1000:], ES1Gt, c = 'r', label = 'GARCH-t')
axs.plot(AOdates[-1000:], ES1GJ, c = 'r', label = 'GJR-GARCH')
axs.plot(AOdates[-1000:], ES1GJt, c = 'cyan', label = 'GJR-GARCH-t')
axs.plot(AOdates[-1000:], ES1HS, c = 'b', label = 'HS')
axs.plot(AOdates[-1000:], ES1RM, c = 'k', label = 'RM')
axs.set_xlim(AOdates[-1000:].min(), AOdates[-1000:].max())
axs.set_title('1% forecast ES from all models with actual returns', weight = 'bold')
axs.legend()
plt.show()

### Q1(d) Assess VaR violations for all models.

In [ ]:
# 95% CI for violation rates
print([0.05 - 1.96 * np.sqrt(0.05 * 0.95 / 1000), 0.05 + 1.96 * np.sqrt(0.05 * 0.95 / 1000) ] )
print([0.01 - 1.96 * np.sqrt(0.01 * 0.99 / 1000), 0.01 + 1.96 * np.sqrt(0.01 * 0.99 / 1000) ] )

#accuracy of violation rates
# TODO: assess VaR violation


The HS method is closest to the nominal violation rate at both 5% and 1%! Among GARCH-type models, RiskMetrics is closest to the 5% rate and GARCH-t to the 1% rate.

In [ ]:
viol5G = ( AOrf<VaR5G )
viol5GJ = ( AOrf<VaR5GJ )
viol5Gt = ( AOrf<VaR5Gt )
viol5GJt = ( AOrf<VaR5GJt )
viol5RM = ( AOrf<VaR5RM )
viol5HS = ( AOrf<VaR5HS )

## p = 0.05 ES residuals (magnitudes of VaR violations)
err5G = ( AOrf - ES5G )[ viol5G ]
err5GJ = ( AOrf - ES5GJ )[ viol5GJ ]
err5Gt = ( AOrf - ES5Gt )[ viol5Gt]
err5GJt = ( AOrf - ES5GJt )[ viol5GJt ]
err5RM = ( AOrf - ES5RM )[ viol5RM ]
err5HS = ( AOrf - ES5HS )[ viol5HS]

#Plots of ES residuals for p=0.05
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(err5G.values, c = 'y', label = 'GARCH')
axs.plot(err5Gt.values, c = 'm', label = 'GARCH-t')
axs.plot(err5GJ.values, c = 'r', label = 'GJR GARCH')
axs.plot(err5GJt.values, c = 'c', label = 'GJR GARCH-t')
axs.plot(err5HS.values, c = 'b', label = 'HS')
axs.plot(err5RM.values, c = 'k', label = 'RM')
axs.set_title('ES Residuals from all models at p=0.05', weight = 'bold')
axs.legend()
plt.show()


In [ ]:
viol1G = ( AOrf<VaR1G )
viol1GJ = ( AOrf<VaR1GJ )
viol1Gt = ( AOrf<VaR1Gt )
viol1GJt = ( AOrf<VaR1GJt )
viol1RM = ( AOrf<VaR1RM )
viol1HS = ( AOrf<VaR1HS )

## p = 0.01 ES residuals (magnitudes of VaR violations)
err1G = ( AOrf - ES1G )[ viol1G ]
err1GJ = ( AOrf - ES1GJ )[ viol1GJ ]
err1Gt = ( AOrf - ES1Gt )[ viol1Gt]
err1GJt = ( AOrf - ES1GJt )[ viol1GJt ]
err1RM = ( AOrf - ES1RM )[ viol1RM ]
err1HS = ( AOrf - ES1HS )[ viol1HS]

#Plots of ES residuals for p=0.01
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(err1G.values, c = 'y', label = 'GARCH')
axs.plot(err1Gt.values, c = 'm', label = 'GARCH-t')
axs.plot(err1GJ.values, c = 'r', label = 'GJR GARCH')
axs.plot(err1GJt.values, c = 'c', label = 'GJR GARCH-t')
axs.plot(err1HS.values, c = 'b', label = 'HS')
axs.plot(err1RM.values, c = 'k', label = 'RM')
axs.set_title('ES Residuals from all models at p=0.01', weight = 'bold')
axs.legend()
plt.show()


In [ ]:
## p = 0.05 standardised ES residuals (scaled by volatility)
err5Gs = ( ( AOrf - ES5G ) / SFAOg )[viol5G]
err5GJs = ( ( AOrf - ES5GJ ) / SFAOgj )[viol5GJ]
err5Gts = ( ( AOrf - ES5Gt ) / SFAOgt )[viol5Gt]
err5GJts = ( ( AOrf - ES5GJt ) / SFAOgjt )[viol5GJt]
err5RMs = ( ( AOrf - ES5RM ) / SFAOrm )[viol5RM]
err5HSs = ( ( AOrf - ES5HS ) / SFAOrm )[viol5HS]

#Plots of ES residuals for p=0.05
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(err5Gs.values, c = 'y', label = 'GARCH')
axs.plot(err5Gts.values, c = 'm', label = 'GARCH-t')
axs.plot(err5GJs.values, c = 'r', label = 'GJR GARCH')
axs.plot(err5GJts.values, c = 'c', label = 'GJR GARCH-t')
axs.plot(err5HSs.values, c = 'b', label = 'HS')
axs.plot(err5RMs.values, c = 'k', label = 'RM')
axs.set_title('Standardised ES Residuals from all models at p=0.05' )
axs.legend()
plt.show()

In [ ]:
## p = 0.01 standardised ES residuals (scaled by volatility)
err1Gs = ( ( AOrf - ES1G ) / SFAOg )[viol1G]
err1GJs = ( ( AOrf - ES1GJ ) / SFAOgj )[viol1GJ]
err1Gts = ( ( AOrf - ES1Gt ) / SFAOgt )[viol1Gt]
err1GJts = ( ( AOrf - ES1GJt ) / SFAOgjt )[viol1GJt]
err1RMs = ( ( AOrf - ES1RM ) / SFAOrm )[viol1RM]
err1HSs = ( ( AOrf - ES1HS ) / SFAOrm )[viol1HS]

#Plots of ES residuals for p=0.01
fig, axs = plt.subplots(figsize = (10,8))
axs.plot(err1Gs.values, c = 'y', label = 'GARCH')
axs.plot(err1Gts.values, c = 'm', label = 'GARCH-t')
axs.plot(err1GJs.values, c = 'r', label = 'GJR GARCH')
axs.plot(err1GJts.values, c = 'c', label = 'GJR GARCH-t')
axs.plot(err1HSs.values, c = 'b', label = 'HS')
axs.plot(err1RMs.values, c = 'k', label = 'RM')
axs.set_title('Standardised ES Residuals from all models at p=0.01' )
axs.legend()
plt.show()

In [ ]:
#For p=0.05: sums, means and t-stats for ES residuals as well as means and t-stats for standardised ES residuals
print('\n', np.array([[np.sum(viol5G), np.sum(viol5GJ), np.sum(viol5RM), np.sum(viol5Gt), np.sum(viol5GJt), np.sum(viol5HS)], 
               [np.mean(err5G), np.mean(err5GJ), np.mean(err5RM), np.mean(err5Gt), np.mean(err5GJt), np.mean(err5HS)], 
               [np.mean(err5G)/(np.std(err5G)/np.sqrt(len(err5G))), np.mean(err5GJ)/(np.std(err5GJ)/np.sqrt(len(err5GJ))), np.mean(err5RM)/(np.std(err5RM)/np.sqrt(len(err5RM))), np.mean(err5Gt)/(np.std(err5Gt)/np.sqrt(len(err5Gt))), np.mean(err5GJt)/(np.std(err5GJt)/np.sqrt(len(err5GJt))), np.mean(err5HS)/(np.std(err5HS)/np.sqrt(len(err5HS)))], 
               [np.mean(err5Gs), np.mean(err5GJs), np.mean(err5RMs), np.mean(err5Gts), np.mean(err5GJts), np.mean(err5HSs)], 
               [np.mean(err5Gs)/(np.std(err5Gs)/np.sqrt(len(err5Gs))), np.mean(err5GJs)/(np.std(err5GJs)/np.sqrt(len(err5GJs))), np.mean(err5RMs)/(np.std(err5RMs)/np.sqrt(len(err5RMs))), np.mean(err5Gts)/(np.std(err5Gts)/np.sqrt(len(err5Gts))), np.mean(err5GJts)/(np.std(err5GJts)/np.sqrt(len(err5GJts))), np.mean(err5HSs)/(np.std(err5HSs)/np.sqrt(len(err5HSs)))]]))

#For p=0.01: sums, means and t-stats for ES residuals as well as means and t-stats for standardised ES residuals
print('\n', np.array([[np.sum(viol1G), np.sum(viol1GJ), np.sum(viol1RM), np.sum(viol1Gt), np.sum(viol1GJt), np.sum(viol1HS)], 
               [np.mean(err1G), np.mean(err1GJ), np.mean(err1RM), np.mean(err1Gt), np.mean(err1GJt), np.mean(err1HS)], 
               [np.mean(err1G)/(np.std(err1G)/np.sqrt(len(err1G))), np.mean(err1GJ)/(np.std(err1GJ)/np.sqrt(len(err1GJ))), np.mean(err1RM)/(np.std(err1RM)/np.sqrt(len(err1RM))), np.mean(err1Gt)/(np.std(err1Gt)/np.sqrt(len(err1Gt))), np.mean(err1GJt)/(np.std(err1GJt)/np.sqrt(len(err1GJt))), np.mean(err1HS)/(np.std(err1HS)/np.sqrt(len(err1HS)))], 
               [np.mean(err1Gs), np.mean(err1GJs), np.mean(err1RMs), np.mean(err1Gts), np.mean(err1GJts), np.mean(err1HSs)], 
               [np.mean(err1Gs)/(np.std(err1Gs)/np.sqrt(len(err1Gs))), np.mean(err1GJs)/(np.std(err1GJs)/np.sqrt(len(err1GJs))), np.mean(err1RMs)/(np.std(err1RMs)/np.sqrt(len(err1RMs))), np.mean(err1Gts)/(np.std(err1Gts)/np.sqrt(len(err1Gts))), np.mean(err1GJts)/(np.std(err1GJts)/np.sqrt(len(err1GJts))), np.mean(err1HSs)/(np.std(err1HSs)/np.sqrt(len(err1HSs)))]]))


Only the GARCH-t, GJR-t models have average ES residuals not significantly different from 0, at both 5% and 1% levels. The GARCH and HS100 models can be rejected for having average 1% ES residuals significantly different from 0. Both models have significantly negative residuals, meaning that their ES forecasts are not extreme enough.

In [ ]:
#RMSE and MAD
print( 'G', 'GJ', 'RM', 'G-t', 'GJ-t', 'HS')
print( np.array([[np.sqrt(np.mean(err5G**2)), np.sqrt(np.mean(err5GJ**2)), np.sqrt(np.mean(err5RM**2)), np.sqrt(np.mean(err5Gt**2)), np.sqrt(np.mean(err5GJt**2)), np.sqrt(np.mean(err5HS**2))], 
          [np.sqrt(np.mean(err1G**2)), np.sqrt(np.mean(err1GJ**2)), np.sqrt(np.mean(err1RM**2)), np.sqrt(np.mean(err1Gt**2)), np.sqrt(np.mean(err1GJt**2)), np.sqrt(np.mean(err1HS**2))], 
          [np.mean(np.abs(err5G)), np.mean(np.abs(err5GJ)), np.mean(np.abs(err5RM)), np.mean(np.abs(err5Gt)), np.mean(np.abs(err5GJt)), np.mean(np.abs(err5HS))], 
          [np.mean(np.abs(err1G)), np.mean(np.abs(err1GJ)), np.mean(np.abs(err1RM)), np.mean(np.abs(err1Gt)), np.mean(np.abs(err1GJt)), np.mean(np.abs(err1HS))]]))

RMSE and MAD are suitable loss functions for forecasting ES accurately. Overall the GARCH-t model appears best under these criteria.

In [ ]:
## For the Gaussian error models:
##  - the ES 5% occurs at the 0.0196 (or 1.96%) quantile,
##  - the ES 1% occurs at the 0.0038 (or 0.38%) quantile.
delaN5 = stats.norm.cdf( ES_n( 0.05 ) )
delaN1 = stats.norm.cdf( ES_n( 0.01 ) )

## For the Student-t error models, use the average degree of freedom from G-t and GJR-t 
## to determine what quantile the 5% and 1% ES should occur at, for that Student-t:
dfGt_avg = np.mean(dfGt)
dfGJt_avg = np.mean(dfGJt)
delaG5 = stats.t.cdf( ES_t( 0.05, dfGt_avg ), dfGt_avg )
delaG1 = stats.t.cdf( ES_t( 0.01, dfGt_avg ), dfGt_avg )
delaGJ5 = stats.t.cdf( ES_t( 0.05, dfGJt_avg ), dfGJt_avg )
delaGJ1 = stats.t.cdf( ES_t( 0.01, dfGJt_avg ), dfGJt_avg )

print( ' G      ', 'GJ     ', 'RM     ', 'G-t    ', 'GJ-t   ', 'HS     ' )
#Set print options to 4dp and suppress scientific notation
np.set_printoptions(precision=5, suppress=True) 
print( np.array( [delaN5, delaN5, delaN5, delaG5, delaGJ5, (delaG5+delaGJ5)/2] ) ) 
print( np.array( [delaN1, delaN1, delaN1, delaG1, delaGJ1, (delaG1+delaGJ1)/2] ) )

#dfGt = np.array(dfGt)
#dfGJt = np.array(dfGJt)
#delaG5t = stats.t.cdf( ES_t( 0.05, dfGt ), dfGt )
#delaG1t = stats.t.cdf( ES_t( 0.01, dfGt ), dfGt )
#delaGJ5t = stats.t.cdf( ES_t( 0.05, dfGJt ), dfGJt )
#delaGJ1t = stats.t.cdf( ES_t( 0.01, dfGJt ), dfGJt )

### Plot Changes in 1% ES quantile estimate
#fig, axs = plt.subplots(figsize = (10,8))
#axs.plot(AOdates[-1000:], delaG1t, c = 'b', label = 'GARCH-t')
#axs.plot(AOdates[-1000:], delaGJ1t, c = 'r', label = 'GJR-GARCH-t')
#axs.set_title('Changes in 1% ES quantile estimate for GARCH-t and GJR-t', weight = 'bold')
#plt.show()


The ES 5% and ES 1% occur at higher quantiles for the Student-t than for the Gaussian.

In [ ]:
#Violations of ES forecasts
viol5VG = (AOrf<ES5G)
viol5VGJ = (AOrf<ES5GJ)
viol5VRM =(AOrf<ES5RM)
viol5VGt = (AOrf<ES5Gt)
viol5VGJt = (AOrf<ES5GJt)
viol5VHS = (AOrf<ES5HS)
viol1VG = (AOrf<ES1G)
viol1VGJ = (AOrf<ES1GJ)
viol1VRM = (AOrf<ES1RM)
viol1VGt = (AOrf<ES1Gt)
viol1VGJt = (AOrf<ES1GJt)
viol1VHS = (AOrf<ES1HS)

#Binomial confidence intervals for Gaussian and t models at 5% and 1% 
def binomial_ci( p ):
    return np.array( [ p - 1.96*np.sqrt( p*(1-p)/1000 ), p + 1.96*np.sqrt( p*(1-p)/1000 ) ] )

print('Gaussian 5% ', binomial_ci( delaN5 ) )
print('Student-t 5%', binomial_ci( delaG5 ) )
print('Gaussian 1% ', binomial_ci( delaN1 ) )
print('Student-t 1%', binomial_ci( delaG1 ) )

## Expect ES 5% violation rates for the Gaussian models in the range (0.0110, 0.0282) 
## with 95% confidence.
## Expect higher ES 5% violation rates for the Student-t models, in the range (0.0178, 0.0382)
## with 95% confidence.

## ES 5% violations 
print( [np.sum(viol5VG), np.sum(viol5VGJ), np.sum(viol5VRM), np.sum(viol5VGt), np.sum(viol5VGJt), np.sum(viol5VHS)] )

## ES 5% violation rates
print( [np.sum(viol5VG)/1000, np.sum(viol5VGJ)/1000, np.sum(viol5VRM)/1000, np.sum(viol5VGt)/1000, np.sum(viol5VGJt)/1000, np.sum(viol5VHS)/1000] )

## ES 1% violations 
print( [np.sum(viol1VG), np.sum(viol1VGJ), np.sum(viol1VRM), np.sum(viol1VGt), np.sum(viol1VGJt), np.sum(viol1VHS)] )

## ES 1% violation rates
print( [np.sum(viol1VG)/1000, np.sum(viol1VGJ)/1000, np.sum(viol1VRM)/1000, np.sum(viol1VGt)/1000, np.sum(viol1VGJt)/1000, np.sum(viol1VHS)/1000] )



The Gaussian models have too many violations, both at 5% and 1%, and can be rejected as under-estimating 5% ES risk levels. The Student-t models agree well with expected violation rates, both at 5% and 1%.

## Q1(e) Assess the models for ES forecast accuracy, independence and loss function values. 

In [ ]:
#Define the 3 functions introduced in tutorial 11.
def indtest(hits):
    n = len(hits)

    r5 = hits[1:]
    r51 = hits[:-1]
    i11 = r5*r51
    i01 = r5*(1-r51)
    i10 = (1-r5)*r51
    i00 = (1-r5)*(1-r51)

    t00 = np.sum(i00)
    t01 = np.sum(i01)
    t10 = np.sum(i10)
    t11 = np.sum(i11)
    p01 = t01/(t00+t01)
    p11 = t11/(t10+t11)
    p1 = (t01+t11)/n

    ll1 = t00 * np.log(1-p01) + (p01>0) * t01 * np.log(p01) + t10 * np.log(1-p11)
    if p11>0:
        ll1=ll1+t11*np.log(p11)
  
    ll0=(t10+t00)*np.log(1-p1)+(t01+t11)*np.log(p1)

    lrind=2*(ll1-ll0)
    pcc=1-stats.chi2.cdf(lrind,1)
    return pcc, lrind

def dqtest(y,f,a,lag):
    n = len(y)
    hits = ((y<f)*1)*(1-a)
    hits = (hits)*1+(y>f)*(-a)
    q=2+lag
    
    if np.sum((y<f)*1) > 0:
        ns = n - lag
        xmat = np.column_stack([np.ones((ns,1)), f[lag:n+1]])
        for k in range(1,lag+1):
            lk = lag-k
            xmat = np.column_stack([xmat, hits[lk:n-k]])
    
        hx = np.dot((hits[lag:n+1]), xmat)
        xtx = np.linalg.lstsq(np.matmul(xmat.T, xmat), np.eye(q), rcond = None)[0]
        dq = np.dot(np.dot(hx, xtx), hx.T)
        dq = dq/(a*(1-a))
        pdq = 1 - stats.chi2.cdf(dq,q)
    else:
        pdq = np.nan
        dq = np.nan
    return pdq, dq

def qregloss(q,r,p):
    q = np.array( q )
    x1 = r[r > q]
    x2 = r[r < q]
    f1 = q[r > q]
    f2 = q[r < q]
    qrgl = p * np.sum(x1-f1) + (1-p) * np.sum(f2-x2)
    return qrgl



In [ ]:
#5% Independence tests
pG,lrG = indtest(viol5VG.values)
pGJ,lrGJ = indtest(viol5VGJ.values)
pRM,lrRM = indtest(viol5VRM.values)
pGt,lrGt = indtest(viol5VGt.values)
pGJt,lrGJt = indtest(viol5VGJt.values)
pHS1,lrHS1 = indtest(viol5VHS.values)
print('5% Independence tests', [round(x,4) for x in [pG, pGJ, pRM, pGt, pGJt, pHS1]])


dela5 = [delaN5, delaN5, delaN5, delaG5, delaGJ5, (delaG5+delaGJ5)/2]
dela1 = [delaN1, delaN1, delaN1, delaG1, delaGJ1, (delaG1+delaGJ1)/2]

#5% DQ tests
pdqG,dqG = dqtest(AOrf,ES5G,0.0196,4)
pdqGJ,dqGJ = dqtest(AOrf,ES5GJ,0.0196,4);
pdqRM,dqRM = dqtest(AOrf,ES5RM,0.0196,4);
pdqGt,dqGt = dqtest(AOrf,ES5Gt,delaG5,4);
pdqGJt,dqGJt = dqtest(AOrf,ES5GJt,delaGJ5,4);
pdqHS,dqHS =dqtest(AOrf,ES5HS,dela5[5],4);
print('5% DQ tests', [round(x, 4) for x in [pdqG, pdqGJ, pdqRM, pdqGt, pdqGJt, pdqHS]])

    #1% Independence tests
pG,lrG = indtest(viol1VG.values)
pGJ,lrGJ = indtest(viol1VGJ.values)
pRM,lrRM = indtest(viol1VRM.values)
pGt,lrGt = indtest(viol1VGt.values)
pGJt,lrGJt = indtest(viol1VGJt.values)
pHS1,lrHS1 = indtest(viol1VHS.values)
print('1% Independence tests', [round(x,4) for x in [pG, pGJ, pRM, pGt, pGJt, pHS1]])




#1% DQ tests
pdqG,dqG = dqtest(AOrf,ES1G,0.0038,4)
pdqGJ,dqGJ = dqtest(AOrf,ES1GJ,0.0038,4);
pdqRM,dqRM = dqtest(AOrf,ES1RM,0.0038,4);
pdqGt,dqGt = dqtest(AOrf,ES1Gt,delaG1,4);
pdqGJt,dqGJt = dqtest(AOrf,ES1GJt,delaGJ1,4);
pdqHS,dqHS =dqtest(AOrf,ES1HS,dela1[5],4);
print('1% DQ tests', [round(x, 4) for x in [pdqG, pdqGJ, pdqRM, pdqGt, pdqGJt, pdqHS]])

#5% Quantile Loss functions
loss5G = qregloss(ES5G,AOrf,0.0196)
loss5GJ = qregloss(ES5GJ,AOrf,0.0196)
loss5RM = qregloss(ES5RM,AOrf,0.0196)
loss5Gt = qregloss(ES5Gt,AOrf,delaG5);
loss5GJt = qregloss(ES5GJt,AOrf,delaGJ5)
loss5HS = qregloss(ES5HS,AOrf, (delaG5 + delaGJ5)/2 )
print('5% Loss Functions', [round(x, 4) for x in [loss5G, loss5GJ, loss5RM, loss5Gt, loss5GJt, loss5HS]])

#1% Quantile Loss functions
loss1G = qregloss(ES1G,AOrf ,0.0038)
loss1GJ = qregloss(ES1GJ,AOrf ,0.0038)
loss1RM = qregloss(ES1RM,AOrf ,0.0038)
loss1Gt = qregloss(ES1Gt,AOrf ,delaG1);
loss1GJt = qregloss(ES1GJt,AOrf ,delaGJ1)
loss1HS = qregloss(ES1HS,AOrf ,dela1[5])
print('1% Loss Functions', [round(x, 4) for x in [loss1G, loss1GJ, loss1RM, loss1Gt, loss1GJt, loss1HS]])

The ES forecasts cannot be rejected for any model by the independence test. However, the Gaussian models are rejected by the more powerful DQ test at both 5% and 1%, while the Student-t and HS models are not rejected.

Unexpectedly, the quantile loss function favours the Gaussian models over the Student-t models.
I am not sure why this is.

# Question 2 VaR and ES forecasting, h > 1

### Consider the All Ordinaries Index data from Q1, now using the full sample.

### Q2(a) Using the models from Q1, forecast the VaR and ES at p = 0.05, 0.01, for the last 10-day period in the sample. Did the actual 10-day return fit in with the forecast 10-day return distributions, VaR and ES measures?

In [ ]:
np.random.seed( 0x123123 )

#Q2(a) Forecast 10-day VaR and ES
    #Last 10 days of sample
nc = len(AOr)
ncis = nc-10
AOris = AOr[:ncis]

#Realised 10-day return over last 10 days
AO10 = np.sum(AOr[-10:])

#Define and estimate models
Mdlg = arch_model(y = AOris, mean = 'constant', vol = 'GARCH', p = 1, q = 1, dist = 'gaussian')
Mdlgt = arch_model(y = AOris, mean = 'constant', vol = 'GARCH', p = 1, q = 1, dist = 'studentst')
Mdlgj = arch_model(y = AOris, mean = 'constant', vol = 'GARCH', p = 1, q = 1, o = 1, dist = 'gaussian')
Mdlgjt = arch_model(y = AOris, mean = 'constant', vol = 'GARCH', p = 1, q = 1, o = 1, dist = 'studentst')

EstMdlg = Mdlg.fit( disp='off' )
EstMdlgt = Mdlgt.fit( disp='off' )
EstMdlgj = Mdlgj.fit( disp='off' )
EstMdlgjt = Mdlgjt.fit( disp='off' )


## Simulate 10-day returns.
def step_10_g():
    a1G = EstMdlg.params['alpha[1]']
    a0G = EstMdlg.params['omega']
    b1G = EstMdlg.params['beta[1]']
    p0G = EstMdlg.params['mu']

    sret = np.zeros( ( nsim ) )
    sf = np.zeros( ( nsim ) )
    
    for h in range( 0, 10 ):
        if h == 0:
            sf.fill( np.sqrt( EstMdlg.forecast( horizon = 1 ).variance.values[-1] )[0] )
        else:
            sf = np.sqrt( a0G + a1G * a**2 + b1G * sf**2 ) 
          
        e = np.random.normal( size = nsim )
        a = sf * e 
        sret += p0G + a
    return sret

def step_10_gt():
    a1Gt = EstMdlgt.params['alpha[1]']
    a0Gt = EstMdlgt.params['omega']
    b1Gt = EstMdlgt.params['beta[1]']
    p0Gt = EstMdlgt.params['mu']
    dfGt = EstMdlgt.params['nu']

    sret = np.zeros( ( nsim ) )
    sf = np.zeros( ( nsim ) )

    for h in range( 0, 10 ):
        if h == 0:
            sf.fill( np.sqrt( EstMdlgt.forecast( horizon = 1 ).variance.values[-1] )[0] )
        else:
            sf = np.sqrt( a0Gt + a1Gt * a**2 + b1Gt * sf**2 ) 

        e = np.random.standard_t( df = dfGt, size = nsim ) * np.sqrt( ( dfGt - 2 ) / dfGt )
        a = sf * e 
        sret += p0Gt + a
    return sret
    
def step_10_gj():    
    a1GJ = EstMdlgj.params['alpha[1]']
    a0GJ = EstMdlgj.params['omega']
    b1GJ = EstMdlgj.params['beta[1]']
    p0GJ = EstMdlgj.params['mu']
    g1GJ = EstMdlgj.params['gamma[1]']
    
    sret = np.zeros( ( nsim ) )
    sf = np.zeros( ( nsim ) )

    for h in range( 0, 10 ):
        if h == 0:
            sf.fill( np.sqrt( EstMdlgj.forecast( horizon = 1 ).variance.values[-1] )[0] )
        else:
            sf = np.sqrt( a0GJ + ( a1GJ + g1GJ * ( a < 0 ) ) * a**2 + b1GJ * sf**2 ) 
           
        e = np.random.normal( size = nsim )
        a = sf * e 
        sret += p0GJ + a
    return sret
    
def step_10_gjt():    
    a1GJt = EstMdlgjt.params['alpha[1]']
    a0GJt = EstMdlgjt.params['omega']
    b1GJt = EstMdlgjt.params['beta[1]']
    p0GJt = EstMdlgjt.params['mu']
    g1GJt = EstMdlgjt.params['gamma[1]']
    dfGJt = EstMdlgjt.params['nu'] 

    sret = np.zeros( ( nsim ) )
    sf = np.zeros( ( nsim ) )
    
    for h in range( 0, 10 ):
        if h == 0:
            sf.fill( np.sqrt( EstMdlgt.forecast( horizon = 1 ).variance.values[-1] )[0] )
        else:
            sf = np.sqrt( a0GJt + ( a1GJt + g1GJt * ( a < 0 ) ) * a**2 + b1GJt * sf**2 ) 
          
        e = np.random.standard_t( df = dfGJt, size = nsim ) * np.sqrt( ( dfGJt - 2 ) / dfGJt )
        a = sf * e 
        sret += p0GJt + a
    return sret
 
import time
start_time = time.time()

nsim = 100000
    
sretg = step_10_g()
sretgt = step_10_gt()
sretgj = step_10_gj()    
sretgjt = step_10_gjt()

#Calculate VaRs at 5% and 1% level from nsim simulated 10-day returns
VaR5G10 = np.percentile(sretg, q = 5, interpolation = 'midpoint')
VaR1G10 = np.percentile(sretg, q = 1, interpolation = 'midpoint')
VaR5GJ10 = np.percentile(sretgj, q = 5, interpolation = 'midpoint')
VaR1GJ10 = np.percentile(sretgj, q = 1, interpolation = 'midpoint')
VaR5Gt10 = np.percentile(sretgt, q = 5, interpolation = 'midpoint')
VaR1Gt10 = np.percentile(sretgt, q = 1, interpolation = 'midpoint')
VaR5GJt10 = np.percentile(sretgjt, q = 5, interpolation = 'midpoint')
VaR1GJt10 = np.percentile(sretgjt, q = 1, interpolation = 'midpoint')
print('\nVaRs', [round(x,4) for x in [VaR5G10, VaR5GJ10, VaR5Gt10, VaR5GJt10, VaR1G10, VaR1GJ10, VaR1Gt10, VaR1GJt10]])

#Calculate ES at 5% and 1% level from nsim simulated 10-day returns
ES5G10 = np.mean(sretg[sretg<=VaR5G10])
ES1G10 = np.mean(sretg[sretg<=VaR1G10])
ES5GJ10 = np.mean(sretgj[sretgj<=VaR5GJ10])
ES1GJ10 = np.mean(sretgj[sretgj<=VaR1GJ10])
ES5Gt10 = np.mean(sretgt[sretgt<=VaR5Gt10])
ES1Gt10 = np.mean(sretgt[sretgt<=VaR1Gt10])
ES5GJt10 = np.mean(sretgjt[sretgjt<=VaR5GJt10])
ES1GJt10 = np.mean(sretgjt[sretgjt<=VaR1GJt10])
print('\nExpected Shortfall', [round(x,4) for x in [ES5G10, ES5GJ10, ES5Gt10, ES5GJt10, ES1G10, ES1GJ10, ES1Gt10, ES1GJt10]])

print("Elapsed: %fs." % (time.time() - start_time))


### Q2(b) Plot the 10-day returns distributions from (a) and comment. Test the number of Monte Carlo samples needed to get reliable and repeatable 10-day VaR and ES forecasts. How much time does each run take at this MC sample size? Is that a reasonable or practical amount of time for this problem?

In [ ]:
#Histograms of 10-day returns distributions
plt.hist(sretg, bins = 50)
plt.title('Histogram of simulated 10-day return distribution - GARCH')
plt.show()
plt.hist(sretgt, bins = 50)
plt.title('Histogram of simulated 10-day return distribution - GARCH-t')
plt.show()
plt.hist(sretgj, bins = 50)
plt.title('Histogram of simulated 10-day return distribution - GJR-GARCH')
plt.show()
plt.hist(sretgjt, bins = 50)
plt.title('Histogram of simulated 10-day return distribution - GJR-GARCH-t')
plt.show()

#JB tests for Gaussianity of 10-day returns distributions
test_stat, p_value = stats.jarque_bera(sretg) 
test_stat, p_value = stats.jarque_bera(sretgj)
test_stat, p_value = stats.jarque_bera(sretgt)
test_stat, p_value = stats.jarque_bera(sretgjt)

#Skewness and Kurtosis of 10-day returns distributions
print([[scipy.stats.skew(sretg), scipy.stats.kurtosis(sretg)], 
       [scipy.stats.skew(sretgj), scipy.stats.kurtosis(sretgj)], 
       [scipy.stats.skew(sretgt), scipy.stats.kurtosis(sretgt)], 
       [scipy.stats.skew(sretgjt), scipy.stats.kurtosis(sretgjt)]])

#RiskMetrics
sigRM = [np.std(AOr[:250])]
for t in range(1, ncis,1):
    sigRM.append(np.sqrt(0.06*AOr.iloc[t-1]**2+0.94*sigRM[t-1]**2))
sigRMf = sigRM[-1]
VaR5RM10 = np.sqrt(10)*sigRM[-1]*stats.norm.ppf(0.05);
VaR1RM10 = np.sqrt(10)*sigRM[-1]*stats.norm.ppf(0.01);
ES5RM10 = -stats.norm.pdf(stats.norm.ppf(0.05),0,1)/0.05*np.sqrt(10)*sigRMf
ES1RM10 = -stats.norm.pdf(stats.norm.ppf(0.01),0,1)/0.01*np.sqrt(10)*sigRMf


print('\nRiskMetrics VaRs', [round(x,4) for x in [VaR1RM10, VaR5RM10]])
print('\nRiskMetrics Expected Shortfalls', [round(x,4) for x in [ES1RM10, ES5RM10]])

The scale of the four distributions is quite comparable. Note that the GJR model induces a small amount of negative skewness in the 10-day return distribution. The skewness for the GJR model are both around -0.47. Also the excess kurtosis for the four distributions are all positive, indicating heavier tails than for a Gaussian random variable. JB tests strongly rejected Gaussianity for all distributions.

I used a Monte Carlo sample size of 100000 and it takes about 0.3s to run each time.